In [1]:
from scrapybara import Scrapybara
from web_recorder import Recorder, Recording

In [2]:
SCRAPYBARA_API_KEY = "<api_key>"
client = Scrapybara(api_key=SCRAPYBARA_API_KEY)

instance = client.start_browser()

live_web_url = instance.get_stream_url().stream_url
cdp_url = instance.get_cdp_url().cdp_url
print(f"`🔍 Live View Link: {live_web_url}")

`🔍 Live View Link: https://api.proxy.scrapybara.com/v1/instance/s-cc2b3608/stream


In [3]:
cdp_url

'wss://proxy-api-102101747085.us-central1.run.app/v1/instance/s-cc2b3608/browser/ws'

**Record a session**

This will start recording your browser session. The recorder will capture all page loads, dom changes, and user interactions, including:
- Mouse clicks
- Mouse movements
- Scrolling
- Text input
- Page navigation
- Page loads

To stop recording, simply close the browser tab that was opened.

In [4]:
recorder = Recorder(
  cdp_url=cdp_url,
)

recording = await recorder.record()

Task ID: d30ad367-0dd0-4dc8-8dbf-8622764b5ae6
Recording completed


In [5]:
events_path = "recordings/events.json"

**Export recording**

Export the recording to a file. 

The export path can be:
- Local file path (e.g. "recordings/events.json") 
- S3 path (e.g. "s3://bucket/path/to/file.json")

In [6]:
await recording.export(
  path=events_path,
)

Successfully exported recording


**Load recording from file**

Load a previously exported recording from a file.

The file path can be:
- Local file path (e.g. "recordings/events.json")
- S3 path (e.g. "s3://bucket/path/to/file.json")


In [3]:
recording = Recording.from_file(events_path)

In [7]:
len(recording.events)

197

**Replay recording**

Replay the recording in a browser. By default it uses a local browser instance.

You can pass in a CDP URL to replay on a remote browser instance.

Example:
```python
# Remote replay
await recording.replay(cdp_url="ws://remote-browser:9222")
```


In [ ]:
await recording.replay()

Replay completed


**Export session as trajectory**

Export the recording as a trajectory format that captures puts out dom for each action from the session.
This is helpful for:
- Analyzing user interactions and page states over time
- Training ML models on web interactions
- Debugging issues by examining the DOM state at each action
- Creating structured data from web sessions

Schema:

```json
{
  "id": "123",
  "snapshots": [
    {
      "action": "page_load",
      "timestamp": 100,
      "state": "<html>...</html>",
      "metadata": {
        "url": "https://example.com"
      },
    },
    {
      "action": "click",
      "timestamp": 101,
      "state": "<html>...</html>",
      "metadata": { ... },
      "element": "<button>...</button>"
    },
    ...
  ]
}


Generate and use trajectory

In [9]:
events_path = "recordings/events.json"
recording = Recording.from_file(events_path)

trajectory = await recording.get_trajectory()

len(trajectory.snapshots)

Generate and export to a path

In [10]:
from web_recorder.recorder import ExportFormat, ExportConfig
events_path = "recordings/events.json"
recording = Recording.from_file(events_path)

await recording.export(
    path="./trajectories/trajectory.json",
    config=ExportConfig(
        format=ExportFormat.TRAJECTORY,
    ),
)

Successfully exported recording
